In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
df = pd.read_csv("data.csv", dtype={'date':np.string_, 'time':np.string_, 'Usage':np.float64})

#dff = pd.DataFrame("data.csv",columns = ['date','time','usage'])

# Set index
df = df.set_index('usage')

# View first few rows
df.head()



TypeError: Empty 'DataFrame': no numeric data to plot